In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=04a1a171d81a85d2ca056744e0773281e44457f7f6dace1f68236e755e299ea0
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=58e4ab0fd128fa3d7c78b18a949c2266d34bd9fad3a92432cee01e711c2e1b78
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
# import fashion_mnist data 

fashion_mnist = keras.datasets.fashion_mnist

In [4]:
# Create training and testing data

(train_images, train_label),(test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
# scaling down the images between 0-1 (all pixels are scaled down between 0 to 1)

train_images = train_images/255.0 # max no. of pixels that we can have
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [0]:
# reshaping the images

train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
train_images[0].shape

(28, 28, 1)

In [0]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int("conv_1_filter", min_value = 32, max_value = 128, step = 16),
        kernel_size = hp.Choice("conv_1_filter", values = [3,5]), #kernal = filter size ie: 3x3 or 4x4
        activation = "relu",
        padding = "same",
        input_shape = (28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int("conv_2_filter", min_value = 32, max_value = 128, step = 16),
        kernel_size = hp.Choice("conv_2_filter", values = [3,5]),
        activation = "relu",
        padding = "same"
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units = hp.Int("dense_1_unit", min_value = 32, max_value = 128, step = 16),
        activation = "relu",
    ),
    keras.layers.Dense(10, activation = "softmax")
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3])),
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
# Run the Randomsearch
# The max_trail runs only for 2 epochs, and tries to find the best hyperparameters

tuner_search = RandomSearch(build_model, objective = "val_accuracy", max_trials = 5, directory = "C:/Users/Ashish/Desktop/Deep Learning/CSV files/Cnn_in_colab", project_name = "Mnist_fashion")

In [12]:
tuner_search.search(train_images, train_label, epochs = 3, validation_split = 0.1)

Epoch 1/3
 168/1688 [=>............................] - ETA: 22:38 - loss: 292.8321 - accuracy: 0.0982

KeyboardInterrupt: ignored

In [0]:
# select the best model

model = tuner_search.get_best_models(num_models = 1)[0]

In [0]:
# Retrain the best model

model.fit(train_images, train_label, epochs = 10, validation_split = 0.1, initial_epochs = 3)

- Initial epochs = 3 coz the model has already been trained for 3 epochs
- The last 3 code did not work coz execution was taking alot of time and hence it was manually interrupted
